In [2]:
import logging
from typing import Dict

import numpy as np
import pandas as pd
import yaml
from scipy.stats.mstats import normaltest
from scipy.stats import boxcox
from sklearn.feature_selection import f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, FunctionTransformer

In [3]:
def remove_unnecessary_columns(
    df: pd.DataFrame,
    delete_columns: Dict
) -> pd.DataFrame:
    """
    Removes unnecessary columns from a DataFrame based on the
    specified criteria.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - delete_columns (Dict): A dictionary containing information
      about columns to delete.
        - 'selected_columns' (List[str]): List of column names to be deleted.
        - 'threshold' (float): Threshold for missing data. Columns with
          missing data exceeding this threshold will be deleted.

    Returns:
    - pd.DataFrame: The DataFrame with unnecessary columns removed.
    """
    df.drop(columns=delete_columns['selected_columns'], inplace=True)

    missing_th = int((1 - delete_columns['threshold']) * len(df)) + 1

    missing_data_cols = [
        col for col in df.columns.tolist() if df[col].count() < missing_th
    ]
    logger = logging.getLogger(__name__)
    if len(missing_data_cols) > 0:
        df.drop(columns=missing_data_cols, inplace=True)
        logger.info("Incomplete deleted columns: ", missing_data_cols)
    else:
        logger.info("There are not deleted columns")

    return df


def remove_incomplete_rows(
    df: pd.DataFrame
) -> pd.DataFrame:
    missing_data_cols = [
        col for col in df.columns.tolist() if len(
            df[pd.isnull(df[col])]
        ) > 0
    ]
    for column in missing_data_cols:
        null_values = df[pd.isnull(df[column])].index.tolist()
        df.drop(null_values, axis=0, inplace=True)
        # print(column, '\t', len(null_values), '\t', df[column].dtypes)
    print("Incomplete deleted rows: ", missing_data_cols)
    return df


def reduce_categorical_column_options(
        df: pd.DataFrame,
        reduce_columns: Dict
) -> pd.DataFrame:
    """
    Reduces the number of options in categorical columns of a DataFrame based
    on specified criteria.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - reduce_columns (Dict): A dictionary containing information about columns
      to be reduced.
        - 'exclude' (List[str]): List of column names to be excluded from
          reduction.
        - 'threshold' (float): Threshold for reducing categories. Categories
          with a frequency below this threshold will be replaced with
          'others'.

    Returns:
    - pd.DataFrame: The DataFrame with reduced categorical options.
    """
    df_data_red = df.copy()
    mask = df.dtypes == object
    categorical_cols = df.columns[mask]
    combined_col_names = []
    for col in categorical_cols:
        if col not in reduce_columns['exclude']:
            val_counts = df[col].value_counts()
            replace_cats = list(
                val_counts[
                    (val_counts / val_counts.sum()) < reduce_columns['threshold']
                ].index
            )
            if len(replace_cats) > 0:
                df_data_red[col] = df_data_red.replace(replace_cats, 'others')[col]
                combined_col_names.append(col)

    if len(combined_col_names) > 0:
        print("Reduced columns: ", combined_col_names)
    else:
        print("Not reduced columns")

    return df_data_red


def handle_outliers(
    df: pd.DataFrame,
    outliers_columns: Dict
) -> pd.DataFrame:
    """
    Handles outliers in specified columns of a DataFrame based on statistical
    tests and boundary criteria.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - outliers_columns (Dict): A dictionary containing information about
      columns with outliers.
        - Keys: Column names with outliers.
        - Values: Not used. Can be an empty dictionary or any placeholder.

    Returns:
    - pd.DataFrame: The DataFrame with outliers removed.
    """
    print("Shape before removing: ", df.shape)
    transformed_columns = []

    for col in outliers_columns:
        p_value = normaltest(df[col].values)[1]
        if p_value < 0.05:
            uppper_boundary = df[col].mean() + 3 * df[col].std()
            lower_boundary = df[col].mean() - 3 * df[col].std()
        else:
            IQR = df[col].quantile(0.75) - df[col].quantile(0.25)
            lower_boundary = df[col].quantile(0.25) - (IQR * 1.5)
            uppper_boundary = df[col].quantile(0.75) + (IQR * 1.5)
        outliers = df[
            (df[col] < lower_boundary) | (df[col] > uppper_boundary)
        ].index.tolist()

        if len(outliers) > 0:
            df.drop(outliers, axis=0, inplace=True)
            transformed_columns.append((col, len(outliers)))

    if len(transformed_columns) > 0:
        print("Outliers deleted: ", transformed_columns)
        print("Shape after removing: ", df.shape)
    else:
        print("There are not outliers")
    return df


def treat_skewed_columns(
    df: pd.DataFrame,
    skewed_columns: Dict
) -> pd.DataFrame:
    """
    Treats skewed numerical columns in a DataFrame using specified
    transformation methods.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - skewed_columns (Dict): A dictionary containing information about columns
      and transformation methods.
        - 'exclude_columns' (List[str]): List of column names to be excluded
          from skewness treatment.
        - 'method' (str): Transformation method. Options: "boxcox" or "log".

    Returns:
    - pd.DataFrame: The DataFrame with treated skewed columns.
    """
    mask_float = df.dtypes == np.float64
    float_cols = df.columns[mask_float].tolist()
    mask_int = df.dtypes == np.int64
    int_cols = df.columns[mask_int].tolist()
    numerical_cols = float_cols + int_cols

    if len(numerical_cols) > 0 and len(skewed_columns['exclude_columns']) > 0:
        for column in skewed_columns['exclude_columns']:
            numerical_cols.remove(column)

    transformed_columns = []
    if skewed_columns['method'] == "boxcox":
        boxcox_dict = {}
        for col in numerical_cols:
            print(col, min(df[col]))
            df[col].fillna(0, inplace=True)
            boxcox_current, lam = boxcox(df[col])
            boxcox_dict.update({f"{col}": [boxcox_current, lam]})
            df[col] = boxcox_current
    elif skewed_columns['method'] == "log":
        for col in numerical_cols:
            p_value = normaltest(df[col].values)[1]
            if p_value > 0.05:
                # print(col, p_value)
                if df[col].min() >= 0:
                    df[col] = (df[col] + 1).transform(np.log)
                else:
                    df[col] = (df[col] - df[col].min() + 1).transform(np.log)
                transformed_columns.append(col)

    if len(transformed_columns) > 0:
        print("Transformed columns: ", transformed_columns)
    else:
        print("There are not transformed columns")

    return df


def encode_categorical_columns(
    df: pd.DataFrame
) -> pd.DataFrame:
    """
    Encodes categorical columns in a DataFrame using various encoding
    techniques.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - target_variable (str): The name of the target variable for binary
      encoding.

    Returns:
    - pd.DataFrame: The DataFrame with encoded categorical columns.
    """
    def transform_education_level(df):
        scale_mapper = {
            "Unknown": 1, "Uneducated": 2, "High School": 3,
            "College": 4, "Graduate": 5, "Post-Graduate": 6, "Doctorate": 7
        }
        df["Education_Level"] = df["Education_Level"].replace(scale_mapper)
        return df


    def transform_income_category(df):
        scale_mapper = {
            "Unknown": 1, "Less than $40K": 2, "$40K - $60K": 3, "$60K - $80K": 4,
            "$80K - $120K": 5, "$120K +": 6
        }
        df["Income_Category"] = df["Income_Category"].replace(scale_mapper)
        return df


    trans_education = FunctionTransformer(transform_education_level)
    trans_income = FunctionTransformer(transform_income_category)

    # Create a ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('Custom_education', trans_education, ['Education_Level']),
            ('Custom_income', trans_income, ['Income_Category']),
            ('MinMax', MinMaxScaler(), ['Customer_Age', 'Months_on_book', 'Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy', 'Total_Trans_Amt']),
            ('Ordinal', OrdinalEncoder(), ['Marital_Status', 'Gender']),
            ('onehot', OneHotEncoder(), ['Card_Category'])
        ],
        remainder='passthrough',  # Leave the other columns unchanged
    )

    # Label encoder
    preprocessor.fit(df)

    return preprocessor


def _get_anova_fvalue(
    x: pd.DataFrame,
    y: pd.Series
) -> pd.DataFrame:
    # Entre mayor sea el f1, quiere decir que la media entre las
    # clases 0 y 1 de attrition, tiene una mayor variabilidad,
    # lo que quiere decir que esa variable si importa en en an?lisis
    f_scores = f_classif(x, y)[0]  # el [1] son los p-values.
    df_fscores = pd.DataFrame({'features': x.columns, 'score': f_scores})
    df_fscores = df_fscores.sort_values('score', ascending=False)

    return df_fscores


def _get_correlations(
    data: pd.DataFrame,
    threshold: float
) -> pd.DataFrame:
    xcorr = data.corr().abs()
    xcorr = xcorr[xcorr > threshold].fillna(0)
    column1 = []
    column2 = []
    for idx in list(xcorr.index):
        for col in list(xcorr.columns):
            # la matriz es diagonal
            if idx == col:
                break
            if (xcorr.loc[idx, col] != 0):
                column1 = column1 + [idx]
                column2 = column2 + [col]
    df_fcorr = pd.DataFrame({'column1': column1, 'column2': column2})
    return df_fcorr


def _remove_columns_by_correlation(
    x: pd.DataFrame,
    df_most_correlated_cols: pd.DataFrame,
    df_anova_fscores: pd.DataFrame
) -> pd.DataFrame:
    for idx in df_most_correlated_cols.index:
        column1 = df_most_correlated_cols.loc[idx, 'column1']
        column2 = df_most_correlated_cols.loc[idx, 'column2']
        score_column1 = df_anova_fscores.loc[
            df_anova_fscores['features'] == column1, 'score'
        ].ravel()
        score_column2 = df_anova_fscores.loc[
            df_anova_fscores['features'] == column2, 'score'
        ].ravel()
        if score_column1 > score_column2:
            df_most_correlated_cols.loc[idx, 'drop'] = column2
        else:
            df_most_correlated_cols.loc[idx, 'drop'] = column1
    drop_features = list(df_most_correlated_cols['drop'].unique())
    print("removed by correlation: ", drop_features)
    df_removed_columns = x.drop(columns=drop_features, axis=1)
    return df_removed_columns


def _remove_columns_by_fvalue(
    df_clean1: pd.DataFrame,
    df_anova_fscores: pd.DataFrame,
    threshold: float
) -> pd.DataFrame:
    df_anova_fscores = df_anova_fscores[df_anova_fscores['score'] > threshold]
    df_removed_columns = df_clean1[df_anova_fscores['features']]
    return df_removed_columns


def feature_selection_correlation_anova(
    df_encoded_data: pd.DataFrame,
    target: str, threshold: Dict
) -> pd.DataFrame:
    """
    Performs feature selection based on correlation and ANOVA F-value
    criteria.

    Parameters:
    - df_encoded_data (pd.DataFrame): The input DataFrame with encoded
      features.
    - target (str): The name of the target variable.
    - threshold (Dict): A dictionary containing threshold values for
      feature selection.
        - 'corr_threshold' (float): Threshold for correlation coefficient.
        - 'fvalue_threshold' (float): Threshold for ANOVA F-value.

    Returns:
    - pd.DataFrame: The DataFrame with selected features based on
      correlation and ANOVA F-value.
    """
    x = df_encoded_data.drop(columns=[target])
    y = df_encoded_data[target]

    df_anova_fscores = _get_anova_fvalue(x, y)
    df_most_correlated_cols = _get_correlations(
        x, threshold['corr_threshold']
    )
    df_clean1 = _remove_columns_by_correlation(
        x, df_most_correlated_cols, df_anova_fscores
    )
    df_model_input = _remove_columns_by_fvalue(
        df_clean1, df_anova_fscores, threshold['fvalue_threshold']
    )
    df_model_input[target] = y

    return df_model_input

In [4]:
from typing import Dict, Tuple, Any

import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    classification_report, accuracy_score, precision_score,
    recall_score, make_scorer, confusion_matrix
)
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns


def split_data(
    df: pd.DataFrame,
    target_variable: str,
    model_options_lg: Dict
) -> Tuple:
    y = df[target_variable]
    x = df.drop(columns=[target_variable])

    strat_shuf_split = StratifiedShuffleSplit(
        n_splits=1, test_size=model_options_lg['test_size'],
        random_state=model_options_lg['random_state']
    )

    train_idx, test_idx = next(strat_shuf_split.split(x, y))
    x_train = df.loc[train_idx, x.columns]
    y_train = df.loc[train_idx, target_variable]
    x_test = df.loc[test_idx, x.columns]
    y_test = df.loc[test_idx, target_variable]

    return x_train, y_train, x_test, y_test


def train_model(
    x_train: pd.DataFrame,
    y_train: pd.Series,
    model_options_lg: Dict
) -> Any:
    skf = StratifiedKFold(shuffle=True,
                          random_state=model_options_lg['random_state'],
                          n_splits=model_options_lg['n_splits'])

    ss = StandardScaler()

    scoring = {
        'accuracy': make_scorer(accuracy_score),
        'precision': make_scorer(precision_score, average='macro'),
        'recall': make_scorer(recall_score, average='macro'),
        'f1': make_scorer(f1_score, average='macro')
    }

    if model_options_lg['model'] == "LogisticRegression":
        lreg = LogisticRegression()

        estimator = Pipeline([
            # ("polynomial_features", PolynomialFeatures()),
            ("scaler", ss),
            ("logistic_regression", lreg)])

        params = {
            # 'polynomial_features__degree': [1, 2, 3],
            'logistic_regression__penalty': ['l1', 'l2'],
            'logistic_regression__C': [4, 6, 10],
            'logistic_regression__solver': ['liblinear']
        }

    elif model_options_lg['model'] == "SVC":
        svc = SVC()

        estimator = Pipeline([
            # ("polynomial_features", PolynomialFeatures()),
            ("scaler", ss),
            ("svc_classifier", svc)])

        params = {
            # 'polynomial_features__degree': [1, 2,3],
            'svc_classifier__C': [2, 4, 6],
            'svc_classifier__kernel': ['rbf', 'sigmoid']
        }
    elif model_options_lg['model'] == "RandomForest":
        rf = RandomForestClassifier()

        estimator = Pipeline([
            # ("polynomial_features", PolynomialFeatures()),
            ("scaler", ss),
            ("RF_classifier", rf)])

        params = {
            # 'polynomial_features__degree': [1, 2,3],
            'RF_classifier__n_estimators': [350, 400, 450],
            'RF_classifier__max_depth': [None, 20],
            'RF_classifier__warm_start': [True]
        }

    grid = GridSearchCV(
        estimator, params, scoring=scoring, refit='f1', cv=skf, n_jobs=-1
    )
    grid.fit(x_train, y_train)

    return grid


def evaluate_model(
    model: Any,
    x_test: pd.DataFrame,
    y_test: pd.Series
):
    score, params = model.best_score_, model.best_params_
    print("Best score: ", score)
    print("Best params: ", params)
    predictions = model.predict(x_test)
    print(classification_report(y_test, predictions))
    print(model.cv_results_['mean_test_f1'])
    cr = classification_report(y_test, predictions, output_dict=True)
    df_cr = pd.DataFrame(cr).iloc[:-1, :].T
    sns.heatmap(df_cr, annot=True)

    return plt

In [5]:
def load_yaml_file(file_path):
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)
    return data

# Example usage
yaml_file_path = 'C:/Users/luisg/Documents/projects/data_science_bank_churn/conf/base/parameters/data_processing.yml'
yaml_data = load_yaml_file(yaml_file_path)


In [6]:
yaml_data

{'target_variable': 'Attrition_Flag',
 'delete_columns': {'threshold': 0.05, 'selected_columns': ['CLIENTNUM']},
 'reduce_options_columns': {'threshold': 0.03, 'exclude': ['Attrition_Flag']},
 'outliers_columns': ['Customer_Age',
  'Dependent_count',
  'Months_on_book',
  'Total_Relationship_Count',
  'Months_Inactive_12_mon',
  'Contacts_Count_12_mon',
  'Credit_Limit',
  'Total_Revolving_Bal',
  'Avg_Open_To_Buy',
  'Total_Amt_Chng_Q4_Q1',
  'Total_Trans_Amt',
  'Total_Trans_Ct',
  'Total_Ct_Chng_Q4_Q1',
  'Avg_Utilization_Ratio'],
 'skewed_columns': {'method': 'log', 'exclude_columns': []},
 'feature_selection': {'corr_threshold': 0.9, 'fvalue_threshold': 1}}

In [7]:
df_data = pd.read_csv(
    'C:/Users/luisg/Documents/projects/data_science_bank_churn/data/01_raw/BankChurners.csv'
)

In [8]:
df_data.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [10]:
df_data.Marital_Status.unique()

array(['Married', 'Single', 'Unknown', 'Divorced'], dtype=object)

In [16]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CLIENTNUM                 10127 non-null  int64  
 1   Attrition_Flag            10127 non-null  object 
 2   Customer_Age              10127 non-null  int64  
 3   Gender                    10127 non-null  object 
 4   Dependent_count           10127 non-null  int64  
 5   Education_Level           10127 non-null  object 
 6   Marital_Status            10127 non-null  object 
 7   Income_Category           10127 non-null  object 
 8   Card_Category             10127 non-null  object 
 9   Months_on_book            10127 non-null  int64  
 10  Total_Relationship_Count  10127 non-null  int64  
 11  Months_Inactive_12_mon    10127 non-null  int64  
 12  Contacts_Count_12_mon     10127 non-null  int64  
 13  Credit_Limit              10127 non-null  float64
 14  Total_

In [17]:
df_data.describe()

,CLIENTNUM,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
count,1.012700e+04,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000
mean,7.391776e+08,46.325960,2.346203,35.928409,3.812580,2.341167,2.455317,8631.953698,1162.814061,7469.139637,0.759941,4404.086304,64.858695,0.712222,0.274894
std,3.690378e+07,8.016814,1.298908,7.986416,1.554408,1.010622,1.106225,9088.776650,814.987335,9090.685324,0.219207,3397.129254,23.472570,0.238086,0.275691
min,7.080821e+08,26.000000,0.000000,13.000000,1.000000,0.000000,0.000000,1438.300000,0.000000,3.000000,0.000000,510.000000,10.000000,0.000000,0.000000
25%,7.130368e+08,41.000000,1.000000,31.000000,3.000000,2.000000,2.000000,2555.000000,359.000000,1324.500000,0.631000,2155.500000,45.000000,0.582000,0.023000
50%,7.179264e+08,46.000000,2.000000,36.000000,4.000000,2.000000,2.000000,4549.000000,1276.000000,3474.000000,0.736000,3899.000000,67.000000,0.702000,0.176000
75%,7.731435e+08,52.000000,3.000000,40.000000,5.000000,3.000000,3.000000,11067.500000,1784.000000,9859.000000,0.859000,4741.000000,81.000000,0.818000,0.503000
max,8.283431e+08,73.000000,5.000000,56.000000,6.000000,6.000000,6.000000,34516.000000,2517.000000,34516.000000,3.397000,18484.000000,139.000000,3.714000,0.999000


In [18]:
for column in df_data.columns:
    print(column, df_data[column].unique())

CLIENTNUM [768805383 818770008 713982108 ... 716506083 717406983 714337233]
Attrition_Flag ['Existing Customer' 'Attrited Customer']
Customer_Age [45 49 51 40 44 32 37 48 42 65 56 35 57 41 61 47 62 54 59 63 53 58 55 66
 50 38 46 52 39 43 64 68 67 60 73 70 36 34 33 26 31 29 30 28 27]
Gender ['M' 'F']
Dependent_count [3 5 4 2 0 1]
Education_Level ['High School' 'Graduate' 'Uneducated' 'Unknown' 'College' 'Post-Graduate'
 'Doctorate']
Marital_Status ['Married' 'Single' 'Unknown' 'Divorced']
Income_Category ['$60K - $80K' 'Less than $40K' '$80K - $120K' '$40K - $60K' '$120K +'
 'Unknown']
Card_Category ['Blue' 'Gold' 'Silver' 'Platinum']
Months_on_book [39 44 36 34 21 46 27 31 54 30 48 37 56 42 49 33 28 38 41 43 45 52 40 50
 35 47 32 20 29 25 53 24 55 23 22 26 13 51 19 15 17 18 16 14]
Total_Relationship_Count [5 6 4 3 2 1]
Months_Inactive_12_mon [1 4 2 3 6 0 5]
Contacts_Count_12_mon [3 2 0 1 4 5 6]
Credit_Limit [12691.  8256.  3418. ...  5409.  5281. 10388.]
Total_Revolving_Bal [ 777  864 

In [19]:
yaml_data

{'target_variable': 'Attrition_Flag',
 'delete_columns': {'threshold': 0.05, 'selected_columns': ['CLIENTNUM']},
 'reduce_options_columns': {'threshold': 0.03, 'exclude': ['Attrition_Flag']},
 'outliers_columns': ['Customer_Age',
  'Dependent_count',
  'Months_on_book',
  'Total_Relationship_Count',
  'Months_Inactive_12_mon',
  'Contacts_Count_12_mon',
  'Credit_Limit',
  'Total_Revolving_Bal',
  'Avg_Open_To_Buy',
  'Total_Amt_Chng_Q4_Q1',
  'Total_Trans_Amt',
  'Total_Trans_Ct',
  'Total_Ct_Chng_Q4_Q1',
  'Avg_Utilization_Ratio'],
 'skewed_columns': {'method': 'log', 'exclude_columns': []},
 'feature_selection': {'corr_threshold': 0.9, 'fvalue_threshold': 1}}

### Data processing

In [20]:
def transform_target(df, target_variable):
    df[target_variable] = df[target_variable].apply(
        lambda x: 0 if x == "Existing Customer" else 1
    )
    df.rename(columns={target_variable: "Attrition"}, inplace=True)
    return df


df_data = pd.read_csv(
    'C:/Users/luisg/Documents/projects/data_science_bank_churn/data/01_raw/BankChurners.csv'
)
print(len(df_data.columns))
df_data = transform_target(df_data, 'Attrition_Flag')
df_data = remove_unnecessary_columns(df_data, yaml_data['delete_columns'])
print(len(df_data.columns))
df_data = reduce_categorical_column_options(df_data, yaml_data['reduce_options_columns'])
df_data = handle_outliers(df_data, yaml_data['outliers_columns'])
# df_data = treat_skewed_columns(df_data, yaml_data['skewed_columns'])
# df_data = encode_categorical_columns(df_data, yaml_data['target_variable'])
df_data.reset_index(inplace=True, drop=True)

21
20
Reduced columns:  ['Card_Category']
Shape before removing:  (10127, 20)
Outliers deleted:  [('Customer_Age', 1), ('Months_Inactive_12_mon', 124), ('Contacts_Count_12_mon', 620), ('Total_Amt_Chng_Q4_Q1', 159), ('Total_Trans_Amt', 308), ('Total_Trans_Ct', 7), ('Total_Ct_Chng_Q4_Q1', 92)]
Shape after removing:  (8816, 20)


In [21]:
df_data.head()

,Attrition,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,0,44,M,2,Graduate,Married,$40K - $60K,Blue,36,3,1,2,4010.0,1247,2763.0,1.376,1088,24,0.846,0.311
1,0,42,M,5,Uneducated,Unknown,$120K +,Blue,31,5,3,2,6748.0,1467,5281.0,0.831,1201,42,0.680,0.217
2,0,57,F,2,Graduate,Married,Less than $40K,Blue,48,5,2,2,2436.0,680,1756.0,1.190,1570,29,0.611,0.279
3,0,45,F,2,Graduate,Married,Unknown,Blue,37,6,1,2,14470.0,1157,13313.0,0.966,1207,21,0.909,0.080
4,1,62,F,0,Graduate,Married,Less than $40K,Blue,49,2,3,3,1438.3,0,1438.3,1.047,692,16,0.600,0.000


In [22]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8816 entries, 0 to 8815
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Attrition                 8816 non-null   int64  
 1   Customer_Age              8816 non-null   int64  
 2   Gender                    8816 non-null   object 
 3   Dependent_count           8816 non-null   int64  
 4   Education_Level           8816 non-null   object 
 5   Marital_Status            8816 non-null   object 
 6   Income_Category           8816 non-null   object 
 7   Card_Category             8816 non-null   object 
 8   Months_on_book            8816 non-null   int64  
 9   Total_Relationship_Count  8816 non-null   int64  
 10  Months_Inactive_12_mon    8816 non-null   int64  
 11  Contacts_Count_12_mon     8816 non-null   int64  
 12  Credit_Limit              8816 non-null   float64
 13  Total_Revolving_Bal       8816 non-null   int64  
 14  Avg_Open

In [23]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, FunctionTransformer, OrdinalEncoder


df = df_data.copy()
target_variable = 'Attrition'
y = df[target_variable]
x = df.drop(columns=[target_variable])


# Ordinal variables
def transform_education_level(df):
    scale_mapper = {
        "Unknown": 1, "Uneducated": 2, "High School": 3,
        "College": 4, "Graduate": 5, "Post-Graduate": 6, "Doctorate": 7
    }
    df["Education_Level"] = df["Education_Level"].replace(scale_mapper)
    return df


def transform_income_category(df):
    scale_mapper = {
        "Unknown": 1, "Less than $40K": 2, "$40K - $60K": 3, "$60K - $80K": 4,
        "$80K - $120K": 5, "$120K +": 6
    }
    df["Income_Category"] = df["Income_Category"].replace(scale_mapper)
    return df


# trans_education = FunctionTransformer(transform_education_level)
# trans_income = FunctionTransformer(transform_income_category)

education_order = ['Unknown', 'Uneducated', 'High School', 'College', 'Graduate', 'Post-Graduate', 'Doctorate']
income_order = ["Unknown", "Less than $40K", "$40K - $60K", "$60K - $80K", "$80K - $120K", "$120K +"]

# Create a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('Custom_education', OrdinalEncoder(categories=[education_order]), ['Education_Level']),
        ('Custom_income', OrdinalEncoder(categories=[income_order]), ['Income_Category']),
        ('MinMax', MinMaxScaler(), ['Customer_Age', 'Months_on_book', 'Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy', 'Total_Trans_Amt']),
        ('Ordinal', OrdinalEncoder(), ['Marital_Status', 'Gender']),
        ('onehot', OneHotEncoder(), ['Card_Category'])
    ],
    remainder='passthrough', # Leave the other columns unchanged

)

# Label encoder
preprocessor.fit(x) 
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('Custom_education',
                                 OrdinalEncoder(categories=[['Unknown',
                                                             'Uneducated',
                                                             'High School',
                                                             'College',
                                                             'Graduate',
                                                             'Post-Graduate',
                                                             'Doctorate']]),
                                 ['Education_Level']),
                                ('Custom_income',
                                 OrdinalEncoder(categories=[['Unknown',
                                                             'Less than $40K',
                                                             '$40K - $60K',
                                                 

In [24]:
x.head()

,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,44,M,2,Graduate,Married,$40K - $60K,Blue,36,3,1,2,4010.0,1247,2763.0,1.376,1088,24,0.846,0.311
1,42,M,5,Uneducated,Unknown,$120K +,Blue,31,5,3,2,6748.0,1467,5281.0,0.831,1201,42,0.680,0.217
2,57,F,2,Graduate,Married,Less than $40K,Blue,48,5,2,2,2436.0,680,1756.0,1.190,1570,29,0.611,0.279
3,45,F,2,Graduate,Married,Unknown,Blue,37,6,1,2,14470.0,1157,13313.0,0.966,1207,21,0.909,0.080
4,62,F,0,Graduate,Married,Less than $40K,Blue,49,2,3,3,1438.3,0,1438.3,1.047,692,16,0.600,0.000


In [25]:
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('Custom_education',
                                 OrdinalEncoder(categories=[['Unknown',
                                                             'Uneducated',
                                                             'High School',
                                                             'College',
                                                             'Graduate',
                                                             'Post-Graduate',
                                                             'Doctorate']]),
                                 ['Education_Level']),
                                ('Custom_income',
                                 OrdinalEncoder(categories=[['Unknown',
                                                             'Less than $40K',
                                                             '$40K - $60K',
                                                 

In [26]:
x.head()

,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,44,M,2,Graduate,Married,$40K - $60K,Blue,36,3,1,2,4010.0,1247,2763.0,1.376,1088,24,0.846,0.311
1,42,M,5,Uneducated,Unknown,$120K +,Blue,31,5,3,2,6748.0,1467,5281.0,0.831,1201,42,0.680,0.217
2,57,F,2,Graduate,Married,Less than $40K,Blue,48,5,2,2,2436.0,680,1756.0,1.190,1570,29,0.611,0.279
3,45,F,2,Graduate,Married,Unknown,Blue,37,6,1,2,14470.0,1157,13313.0,0.966,1207,21,0.909,0.080
4,62,F,0,Graduate,Married,Less than $40K,Blue,49,2,3,3,1438.3,0,1438.3,1.047,692,16,0.600,0.000


### Model Training

In [27]:

def load_yaml_file(file_path):
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)
    return data

# Example usage
yaml_file_path = 'C:/Users/luisg/Documents/projects/data_science_bank_churn/conf/base/parameters/data_science.yml'
yaml_training_data = load_yaml_file(yaml_file_path)

In [28]:
yaml_training_data

{'target': 'Attrition',
 'model_options_lg': {'n_splits': 3,
  'test_size': 0.2,
  'random_state': 42,
  'model': 'LogisticRegression'},
 'model_options_svm': {'n_splits': 3,
  'test_size': 0.2,
  'random_state': 42,
  'model': 'SVC'},
 'model_options_rf': {'n_splits': 3,
  'test_size': 0.2,
  'random_state': 42,
  'model': 'RandomForest'}}

In [29]:
# x_train, y_train, x_test, y_test = split_data(df_data, 'Attrition', yaml_training_data['model_options_lg'])
# target_variable = 'Attrition'
# y = df_data[target_variable]
# x = df_data.drop(columns=[target_variable])

strat_shuf_split = StratifiedShuffleSplit(
    n_splits=1, test_size=0.2,
    random_state=42
)
# Use the split method to get the indices for training and testing sets
# for train_index, test_index in strat_shuf_split.split(x, y):
#     X_train, X_test = x[train_index], x[test_index]
#     y_train, y_test = y[train_index], y[test_index]
# train_idx, test_idx = next(strat_shuf_split.split(x, y))
# x_train = df_data.loc[train_idx, x.columns]
# y_train = df_data.loc[train_idx, target_variable]
# x_test = df_data.loc[test_idx, x.columns]
# y_test = df_data.loc[test_idx, target_variable]

train_idx, test_idx = next(strat_shuf_split.split(x, y))
x_train = x.iloc[train_idx, :]
y_train = y[train_idx]
x_test = x.loc[test_idx, :]
y_test = y[test_idx]

In [30]:
x_train

,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
7322,36,F,1,Graduate,Married,Less than $40K,Blue,29,4,2,3,1493.0,0,1493.0,0.721,2516,63,0.575,0.000
5747,62,M,0,Graduate,Divorced,$40K - $60K,Blue,36,6,3,2,4049.0,0,4049.0,0.567,4003,73,0.659,0.000
6555,42,M,4,Graduate,Single,$40K - $60K,Blue,33,5,3,2,2654.0,1489,1165.0,0.668,4767,82,0.673,0.561
7892,43,M,2,High School,Single,$120K +,Blue,30,1,2,2,10896.0,2517,8379.0,1.028,4935,58,0.487,0.231
2816,50,F,3,Graduate,Single,Less than $40K,Blue,36,4,3,2,1438.3,1002,436.3,1.306,2004,25,0.667,0.697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4074,42,F,3,Uneducated,Single,Less than $40K,Blue,22,4,1,4,1438.3,860,578.3,0.761,4754,79,0.837,0.598
7739,47,M,2,Graduate,Unknown,$80K - $120K,Blue,41,2,2,2,21325.0,2197,19128.0,0.647,7053,69,0.865,0.103
668,43,F,4,Graduate,Married,Less than $40K,Blue,37,4,3,2,4724.0,2056,2668.0,0.423,1232,31,0.550,0.435
1430,35,M,2,Uneducated,Married,$120K +,Blue,26,3,3,4,2944.0,1596,1348.0,0.737,2070,44,0.467,0.542


In [31]:
skf = StratifiedKFold(shuffle=True, random_state=42, n_splits=3)


scoring = {
    # 'accuracy': make_scorer(accuracy_score),
    # 'precision': make_scorer(precision_score, average='macro'),
    # 'recall': make_scorer(recall_score, average='macro'),
    'f1': make_scorer(f1_score, average='macro')
}

lreg = LogisticRegression()

estimator = Pipeline([
    ("preprocessing", preprocessor),
    ("logistic_regression", lreg)]
)

params = {
    # 'polynomial_features__degree': [1, 2, 3],
    'logistic_regression__penalty': ['l1', 'l2'],
    'logistic_regression__C': [4, 6, 10],
    'logistic_regression__solver': ['liblinear']
}

grid = GridSearchCV(
    estimator, params, scoring=scoring, refit='f1', cv=skf, n_jobs=-1
)
grid.fit(x_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('Custom_education',
                                                                         OrdinalEncoder(categories=[['Unknown',
                                                                                                     'Uneducated',
                                                                                                     'High '
                                                                                                     'School',
                                                                                                     'College',
                                                                                                     'Graduate',
                   

In [32]:
model = grid.best_estimator_

In [33]:
model

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Custom_education',
                                                  OrdinalEncoder(categories=[['Unknown',
                                                                              'Uneducated',
                                                                              'High '
                                                                              'School',
                                                                              'College',
                                                                              'Graduate',
                                                                              'Post-Graduate',
                                                                              'Doctorate']]),
                                                  ['Education_Level']),
                                         

In [34]:
grid.cv_results_

{'mean_fit_time': array([0.15127476, 0.07060917, 0.10956836, 0.07816617, 0.08556056,
        0.05198479]),
 'std_fit_time': array([0.05847052, 0.00289896, 0.02121791, 0.00435732, 0.00534951,
        0.00253826]),
 'mean_score_time': array([0.01641091, 0.01617161, 0.01644508, 0.01636219, 0.01419425,
        0.00838232]),
 'std_score_time': array([0.0001914 , 0.00056389, 0.00432153, 0.00012813, 0.00183975,
        0.0059272 ]),
 'param_logistic_regression__C': masked_array(data=[4, 4, 6, 6, 10, 10],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_logistic_regression__penalty': masked_array(data=['l1', 'l2', 'l1', 'l2', 'l1', 'l2'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_logistic_regression__solver': masked_array(data=['liblinear', 'liblinear', 'liblinear', 'liblinear',
                    'liblinear', 'liblinear'],
              ma

In [35]:
from pathlib import Path

file_path = Path.cwd()
model_path = file_path.parent / "data" / "06_models" / "model_notebook.pkl"
preprocessor_path = file_path.parent / "data" / "06_models" / "preprocessor.pkl"
model_path

WindowsPath('c:/Users/luisg/Documents/projects/data_science_bank_churn/data/06_models/model_notebook.pkl')

In [37]:
import pickle

# Save the model using pickle
with open(model_path, 'wb') as file:
    pickle.dump(model, file)

# Save the preprocessor using pickle
with open(preprocessor_path, 'wb') as file:
    pickle.dump(preprocessor, file)

In [38]:
import pickle


# Now you can load the model later if needed
with open(model_path, 'rb') as file:
    loaded_model = pickle.load(file)

# Now you can load the model later if needed
with open(preprocessor_path, 'rb') as file:
    preprocessor = pickle.load(file)

c:\Users\luisg\Documents\projects\data_science_bank_churn\data\06_models\model_notebook.pkl


In [185]:
x_test

,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
999,56,F,4,High School,Single,Less than $40K,Blue,36,3,1,3,8881.0,2145,6736.0,0.705,2027,49,0.815,0.242
1697,54,F,0,Graduate,Divorced,Less than $40K,Silver,36,4,3,1,12547.0,1378,11169.0,1.120,3360,56,0.750,0.110
653,26,M,0,Graduate,Single,Unknown,Blue,19,4,1,2,1438.3,0,1438.3,0.472,2005,47,0.469,0.000
3863,51,M,2,High School,Married,$60K - $80K,Blue,46,5,3,2,4123.0,1760,2363.0,0.869,2447,40,0.429,0.427
271,46,F,4,Graduate,Divorced,Less than $40K,Blue,36,5,3,1,9478.0,820,8658.0,0.640,1327,41,0.864,0.087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4616,39,F,1,Uneducated,Single,Less than $40K,Blue,34,3,1,4,6904.0,1148,5756.0,0.649,3485,84,0.615,0.166
2784,46,F,2,High School,Unknown,Unknown,Blue,32,4,1,3,7966.0,1423,6543.0,0.987,3780,75,0.923,0.179
3136,43,M,5,Graduate,Unknown,$40K - $60K,Blue,38,3,3,2,7599.0,1100,6499.0,0.365,4401,71,0.651,0.145
1428,37,F,3,Graduate,Single,Less than $40K,Blue,36,5,2,4,3685.0,1396,2289.0,0.974,3016,85,0.809,0.379


In [186]:
predictions = loaded_model.predict(x_test)

In [187]:
y_test

999     0
1697    0
653     0
3863    1
271     0
       ..
4616    0
2784    0
3136    0
1428    0
5635    0
Name: Attrition, Length: 1764, dtype: int64

In [188]:
accuracy_score(y_test, predictions)

0.8979591836734694

In [189]:
y_train

7322    1
5747    0
6555    0
7892    1
2816    0
       ..
4074    0
7739    0
668     0
1430    0
55      0
Name: Attrition, Length: 7052, dtype: int64

In [190]:
f1_score(y_train, loaded_model.predict(x_train))

0.6544930321960596

In [191]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94      1469
           1       0.75      0.59      0.66       295

    accuracy                           0.90      1764
   macro avg       0.83      0.77      0.80      1764
weighted avg       0.89      0.90      0.89      1764



In [192]:
confusion_matrix(y_test, predictions)

array([[1411,   58],
       [ 122,  173]], dtype=int64)

In [4]:
import pandas as pd


df_prueba = pd.read_csv('C:/Users/luisg/Documents/projects/data_science_bank_churn/data/01_raw/BankChurners.csv')
df_prueba = df_prueba.head(1)
df_prueba.drop(columns=['Attrition_Flag'], inplace=True)
# df_prueba.rename(columns={"Attrition_Flag": "Attrition"}, inplace=True)
df_prueba.head()

,CLIENTNUM,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061


In [1]:
print(len(df_prueba.columns))

NameError: name 'df_prueba' is not defined

In [6]:
# df_prueba = remove_unnecessary_columns(df_prueba, yaml_data['delete_columns'])
df_prueba.drop(columns=['CLIENTNUM'], inplace = True)
df_prueba.reset_index(inplace=True, drop=True)
df_prueba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer_Age              1 non-null      int64  
 1   Gender                    1 non-null      object 
 2   Dependent_count           1 non-null      int64  
 3   Education_Level           1 non-null      object 
 4   Marital_Status            1 non-null      object 
 5   Income_Category           1 non-null      object 
 6   Card_Category             1 non-null      object 
 7   Months_on_book            1 non-null      int64  
 8   Total_Relationship_Count  1 non-null      int64  
 9   Months_Inactive_12_mon    1 non-null      int64  
 10  Contacts_Count_12_mon     1 non-null      int64  
 11  Credit_Limit              1 non-null      float64
 12  Total_Revolving_Bal       1 non-null      int64  
 13  Avg_Open_To_Buy           1 non-null      float64
 14  Total_Amt_Chng

In [7]:
len(df_prueba.values.tolist()[0])

19

In [8]:
# df_prueba.drop(columns=['CLIENTNUM'], inplace= True)
# data_scaled=preprocessor.transform(df_prueba)

result_dict = {0: "No Churn", 1: "Churn"}
result = loaded_model.predict(df_prueba)
{'result': result_dict[result[0]]}

{'result': 'No Churn'}

In [239]:
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('Custom_education',
                                 FunctionTransformer(func=<function transform_education_level at 0x0000027BD60BC598>),
                                 ['Education_Level']),
                                ('Custom_income',
                                 FunctionTransformer(func=<function transform_income_category at 0x0000027BD60BC0D0>),
                                 ['Income_Category']),
                                ('MinMax', MinMaxScaler(),
                                 ['Customer_Age', 'Months_on_book',
                                  'Credit_Limit', 'Total_Revolving_Bal',
                                  'Avg_Open_To_Buy', 'Total_Trans_Amt']),
                                ('Ordinal', OrdinalEncoder(),
                                 ['Marital_Status', 'Gender']),
                                ('onehot', OneHotEncoder(), ['Card_Category'])])